In [ ]:
!pip install -q gpt-2-simple


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime


!nvidia-smi

In [ ]:
gpt2.download_gpt2(model_name="774M")
gpt2.download_gpt2(model_name="355M")
gpt2.download_gpt2(model_name="1558M")

In [ ]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip

!unzip python.zip

!gzip -d python/final/jsonl/train/python_train_0.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_1.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_2.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_3.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_4.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_5.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_6.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_7.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_8.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_9.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_10.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_11.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_12.jsonl.gz
!gzip -d python/final/jsonl/train/python_train_13.jsonl.gz

In [ ]:
import pandas as pd
all_sets = []
include = 14
for i in range(include):
  print(i)
  fname = f"python/final/jsonl/train/python_train_{str(i)}.jsonl"
  cur = pd.read_json(fname, lines=True)
  cur2 = cur["code"]
  all_sets.append(cur2)
set1 = pd.concat(all_sets, axis=0)

set1 = set1.to_frame()

set1["length"] = set1["code"].str.len()
set1 = set1[ set1["length"] < 1000]


def find_real_funcs(row):
  word_list = row["code"].split()
  if len(word_list) < 3 or len(word_list[1]) < 2:
    return False
  else:
    if word_list[1][0] == "_":
      return False
    else:
      return True

set1["internal_func"] = set1.apply(find_real_funcs, axis=1)
set1 = set1[ set1["internal_func"]]

def find_return_funcs(row):
  sent_list = row["code"].split("\n")
  if len(sent_list[-1]) == 0:
    return False
  else:
    if "return" in sent_list[-1]:
      return True
    else:
      return False

set1["has_return"] = set1.apply(find_return_funcs, axis=1)
set1 = set1[ set1["has_return"]]
set1 = set1.reset_index(drop=True)

def put_return_first(row):
  code = row["code"].split("\n")
  return_line = code[-1]
  code = code[:-1]
  code.insert(1, return_line)
  code = "\n".join(code)
  return code

set1["updated_code"] = set1.apply( put_return_first, axis=1)
set1 = set1["updated_code"]
full_frame = set1
full_frame = full_frame.astype(str) + "\n <|endoftext|> " 

full_frame.to_csv("python/final/jsonl/train/gptready.txt", index=False, header=False, sep="\n")
full_frame.head()
full_frame.shape

In [ ]:
file_name = "python/final/jsonl/train/gptready.txt"
full_frame = None

sess = gpt2.start_tf_sess()
# 774M
gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=-1,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

In [ ]:
gpt2.load_gpt2(sess, run_name='run1')

In [ ]:
gpt2.generate(sess, run_name='run1')

In [ ]:
gpt2.generate(sess,
              run_name = "run1",
              prefix="def join_text(text_list):\n    return joined_text",
              nsamples=20,
              batch_size=5,
              top_p = 0.9,
              temperature = 0.8
              )